In [7]:
# Tüm bağımlılıkları temizleyip yeniden yükle
!pip uninstall -y numpy TTS
!pip install --upgrade numpy
!pip install TTS
!pip install psutil matplotlib

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: TTS 0.22.0
Uninstalling TTS-0.22.0:
  Successfully uninstalled TTS-0.22.0
  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gruut 2.2.3 requires numpy<2.0.0,>=1.19.0, but you have numpy 2.2.5 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
xarray 2025.3.1 requires pandas>=2.1, but you have pandas 1.5.3 which is incompatible.
scikit-image 0.25.2 requires networkx>=3.0

^C


In [2]:
# -*- coding: utf-8 -*-
"""TTS Model Performance Comparison (GPU vs CPU) - Coqui-TTS.ipynb"""

# Adım 1: Gerekli Kütüphanelerin Yüklenmesi
!pip install -q TTS
!pip install -q psutil matplotlib

import time
import psutil
import torch
from TTS.api import TTS
import pandas as pd
import matplotlib.pyplot as plt

# Adım 2: Donanım Bilgilerini Yazdır
print(f"GPU Kullanılabilir mi? {torch.cuda.is_available()}")
print(f"GPU Adı: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'YOK'}")
print(f"CPU Çekirdek Sayısı: {psutil.cpu_count(logical=True)}")

# Adım 3: Test Modellerini ve Metinleri Tanımla
models_to_test = {
    "Tacotron2": "tts_models/en/ljspeech/tacotron2-DDC",  # İngilizce model
    "VITS": "tts_models/en/ljspeech/vits"  # Daha hızlı bir model
}

test_texts = [
    "The quick brown fox jumps over the lazy dog.",
    "Hello world, this is a performance test.",
    "Deep learning requires massive computational power."
]

# Adım 4: Performans Testi Fonksiyonu (GÜNCELLENDİ)
def run_tts_performance_test(model_name, model_path, device="cuda"):
    print(f"\n🔥 {model_name} ({device.upper()}) Testi Başlıyor...")

    try:
        # Yeni API kullanımı
        tts = TTS(model_path, progress_bar=False).to(device)

        results = []
        for i, text in enumerate(test_texts):
            # Kaynak kullanımı ölçümü
            cpu_percent_before = psutil.cpu_percent(interval=0.1)
            mem_before = psutil.virtual_memory().used / (1024 ** 3)  # GB cinsinden

            # Zaman ölçümü
            start_time = time.time()
            audio = tts.tts(text=text)  # Yeni API'de doğrudan tts() kullanımı
            end_time = time.time()

            # Sonrası kaynak kullanımı
            cpu_percent_after = psutil.cpu_percent(interval=0.1)
            mem_after = psutil.virtual_memory().used / (1024 ** 3)

            duration = end_time - start_time
            results.append({
                "Model": model_name,
                "Cihaz": device,
                "Metin No": i+1,
                "Süre (s)": duration,
                "CPU Kullanımı (%)": (cpu_percent_before + cpu_percent_after) / 2,
                "Bellek Artışı (GB)": mem_after - mem_before
            })

            print(f"Metin {i+1}: {duration:.2f} saniye")

        return pd.DataFrame(results)

    except Exception as e:
        print(f"❌ Hata: {str(e)}")
        return pd.DataFrame()

# Adım 5: Tüm Testleri Çalıştır (GÜNCELLENDİ)
all_results = []
for model_name, model_path in models_to_test.items():
    # GPU testi
    if torch.cuda.is_available():
        print(f"\n=== {model_name} GPU Testi ===")
        gpu_results = run_tts_performance_test(model_name, model_path, "cuda")
        if not gpu_results.empty:
            all_results.append(gpu_results)

    # CPU testi
    print(f"\n=== {model_name} CPU Testi ===")
    cpu_results = run_tts_performance_test(model_name, model_path, "cpu")
    if not cpu_results.empty:
        all_results.append(cpu_results)

# Sonuçları birleştir
if all_results:
    results_df = pd.concat(all_results)

    # Adım 6: Sonuçları Görselleştir
    plt.figure(figsize=(12, 6))

    # Süre Karşılaştırması
    plt.subplot(1, 2, 1)
    for model in models_to_test.keys():
        for device in ["cuda", "cpu"]:
            subset = results_df[(results_df["Model"] == model) & (results_df["Cihaz"] == device)]
            if not subset.empty:
                plt.bar(f"{model} ({device})", subset["Süre (s)"].mean(), label=f"{model} {device}")
    plt.title("Ortalama Ses Üretme Süresi")
    plt.ylabel("Saniye")
    plt.xticks(rotation=45)
    plt.legend()

    # Bellek Kullanımı
    plt.subplot(1, 2, 2)
    for model in models_to_test.keys():
        for device in ["cuda", "cpu"]:
            subset = results_df[(results_df["Model"] == model) & (results_df["Cihaz"] == device)]
            if not subset.empty:
                plt.bar(f"{model} ({device})", subset["Bellek Artışı (GB)"].mean(), label=f"{model} {device}")
    plt.title("Ortalama Bellek Artışı")
    plt.ylabel("GB")
    plt.xticks(rotation=45)

    plt.tight_layout()
    plt.show()

    # Adım 7: Detaylı Sonuç Tablosu
    print("\n📊 Detaylı Performans Metrikleri:")
    display(results_df.groupby(["Model", "Cihaz"]).mean())
else:
    print("❌ Hiçbir test başarıyla tamamlanamadı!")

GPU Kullanılabilir mi? True
GPU Adı: Tesla T4
CPU Çekirdek Sayısı: 2

=== Tacotron2 GPU Testi ===

🔥 Tacotron2 (CUDA) Testi Başlıyor...
 > Downloading model to /root/.local/share/tts/tts_models--en--ljspeech--tacotron2-DDC
 > Model's license - apache 2.0
 > Check https://choosealicense.com/licenses/apache-2.0/ for more info.
 > Downloading model to /root/.local/share/tts/vocoder_models--en--ljspeech--hifigan_v2
 > Model's license - apache 2.0
 > Check https://choosealicense.com/licenses/apache-2.0/ for more info.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:

Metin No   Süre (s)  CPU Kullanımı (%)  Bellek Artışı (GB)
Model     Cihaz                                                            
Tacotron2 cpu         2.0  17.579869           1.600000            0.251155
          cuda        2.0   0.603242           2.500000            0.061993
VITS      cpu         2.0  11.820352           2.466667            0.257589
          cuda        2.0   0.710917          37.866667            0.015816

In [5]:
# -*- coding: utf-8 -*-
"""Enhanced TTS Model Performance Comparison (GPU vs CPU).ipynb"""

# 1. Gerekli Kütüphaneler
!pip install -q TTS gTTS psutil matplotlib pynvml

import time
import psutil
import torch
import pandas as pd
import matplotlib.pyplot as plt
from gtts import gTTS
from io import BytesIO
from TTS.api import TTS

# 2. Sistem Bilgileri
print(f"\n{'='*50}\nSistem Bilgileri\n{'='*50}")
print(f"GPU Kullanılabilir: {torch.cuda.is_available()}")
print(f"GPU Model: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'YOK'}")
print(f"CPU Çekirdek: {psutil.cpu_count(logical=True)}")
print(f"RAM: {psutil.virtual_memory().total / (1024**3):.2f} GB")

# 3. Test Ayarları
models = {
    "Tacotron2": "tts_models/en/ljspeech/tacotron2-DDC",
    "VITS": "tts_models/en/ljspeech/vits",
    "FastSpeech2": "tts_models/en/ljspeech/fastspeech2",
    "GoogleTTS": None  # Özel işlem gerektirir
}

test_texts = [
    "The quick brown fox jumps over the lazy dog",
    "Artificial intelligence is transforming our world",
    "This performance test compares different TTS systems"
]

# 4. Performans Test Fonksiyonları
def get_system_stats():
    return {
        "CPU (%)": psutil.cpu_percent(interval=0.1),
        "RAM (GB)": psutil.virtual_memory().used / (1024 ** 3)
    }

def test_coqui_model(model_name, model_path, device):
    print(f"\n{'='*30}\n{model_name} ({device.upper()}) Testi\n{'='*30}")

    try:
        # Model yükleme
        stats_before = get_system_stats()
        load_start = time.time()

        tts = TTS(model_name=model_path, progress_bar=False).to(device)
        load_time = time.time() - load_start

        # Testler
        results = []
        for i, text in enumerate(test_texts):
            # Öncesi kaynak kullanımı
            stats_before = get_system_stats()
            start_time = time.time()

            # Inference
            audio = tts.tts(text=text)

            # Sonrası ölçümler
            duration = time.time() - start_time
            stats_after = get_system_stats()

            results.append({
                "Model": model_name,
                "Cihaz": device.upper(),
                "Metin No": i+1,
                "Yükleme Süresi": load_time,
                "Süre (s)": duration,
                "CPU (%)": (stats_before["CPU (%)"] + stats_after["CPU (%)"]) / 2,
                "RAM Artışı (GB)": stats_after["RAM (GB)"] - stats_before["RAM (GB)"]
            })

            print(f"Metin {i+1}: {duration:.2f}s | CPU: {results[-1]['CPU (%)']:.1f}% | RAM: {results[-1]['RAM Artışı (GB)']:.2f}GB")

        return pd.DataFrame(results)

    except Exception as e:
        print(f"Hata: {str(e)}")
        return pd.DataFrame()

def test_google_tts():
    print(f"\n{'='*30}\nGoogleTTS Testi\n{'='*30}")

    results = []
    for i, text in enumerate(test_texts):
        stats_before = get_system_stats()
        start_time = time.time()

        # Google TTS işlemi
        mp3_fp = BytesIO()
        tts = gTTS(text=text, lang='en')
        tts.write_to_fp(mp3_fp)

        duration = time.time() - start_time
        stats_after = get_system_stats()

        results.append({
            "Model": "GoogleTTS",
            "Cihaz": "API",
            "Metin No": i+1,
            "Yükleme Süresi": 0,
            "Süre (s)": duration,
            "CPU (%)": (stats_before["CPU (%)"] + stats_after["CPU (%)"]) / 2,
            "RAM Artışı (GB)": stats_after["RAM (GB)"] - stats_before["RAM (GB)"]
        })

        print(f"Metin {i+1}: {duration:.2f}s | CPU: {results[-1]['CPU (%)']:.1f}% | RAM: {results[-1]['RAM Artışı (GB)']:.2f}GB")

    return pd.DataFrame(results)

# 5. Tüm Testleri Çalıştır
all_results = []

# Coqui Modelleri
for name, path in models.items():
    if name == "GoogleTTS":
        continue

    if torch.cuda.is_available():
        gpu_df = test_coqui_model(name, path, "cuda")
        if not gpu_df.empty:
            all_results.append(gpu_df)

    cpu_df = test_coqui_model(name, path, "cpu")
    if not cpu_df.empty:
        all_results.append(cpu_df)

# Google TTS
gtts_df = test_google_tts()
all_results.append(gtts_df)

# 6. Sonuçları Analiz Et
if all_results:
    results_df = pd.concat(all_results)

    # Görselleştirme
    plt.figure(figsize=(18, 6))

    # Süre Karşılaştırması
    plt.subplot(1, 3, 1)
    for model in results_df['Model'].unique():
        for device in results_df[results_df['Model'] == model]['Cihaz'].unique():
            avg_time = results_df[(results_df['Model'] == model) &
                                (results_df['Cihaz'] == device)]['Süre (s)'].mean()
            plt.bar(f"{model}\n({device})", avg_time, label=f"{model} ({device})")
    plt.title("Ortalama İşlem Süresi (saniye)")
    plt.ylabel("Süre (s)")
    plt.xticks(rotation=45)
    plt.grid(True, linestyle='--', alpha=0.6)

    # CPU Kullanımı
    plt.subplot(1, 3, 2)
    for model in results_df['Model'].unique():
        for device in results_df[results_df['Model'] == model]['Cihaz'].unique():
            avg_cpu = results_df[(results_df['Model'] == model) &
                               (results_df['Cihaz'] == device)]['CPU (%)'].mean()
            plt.bar(f"{model}\n({device})", avg_cpu, color='orange')
    plt.title("Ortalama CPU Kullanımı (%)")
    plt.ylabel("CPU (%)")
    plt.xticks(rotation=45)
    plt.grid(True, linestyle='--', alpha=0.6)

    # Bellek Kullanımı
    plt.subplot(1, 3, 3)
    for model in results_df['Model'].unique():
        for device in results_df[results_df['Model'] == model]['Cihaz'].unique():
            avg_ram = results_df[(results_df['Model'] == model) &
                               (results_df['Cihaz'] == device)]['RAM Artışı (GB)'].mean()
            plt.bar(f"{model}\n({device})", avg_ram, color='green')
    plt.title("Ortalama RAM Artışı (GB)")
    plt.ylabel("RAM (GB)")
    plt.xticks(rotation=45)
    plt.grid(True, linestyle='--', alpha=0.6)

    plt.tight_layout()
    plt.show()

    # Detaylı Sonuçlar
    print("\n" + "="*50)
    print("Detaylı Performans Sonuçları")
    print("="*50)
    display(results_df.groupby(['Model', 'Cihaz']).mean().style
           .background_gradient(cmap='Blues', subset=['Süre (s)'])
           .background_gradient(cmap='Oranges', subset=['CPU (%)'])
           .background_gradient(cmap='Greens', subset=['RAM Artışı (GB)'])
           .format("{:.2f}"))
else:
    print("Hiçbir test başarıyla tamamlanamadı!")


Sistem Bilgileri
GPU Kullanılabilir: True
GPU Model: Tesla T4
CPU Çekirdek: 2
RAM: 12.67 GB

Tacotron2 (CUDA) Testi
 > tts_models/en/ljspeech/tacotron2-DDC is already downloaded.
 > vocoder_models/en/ljspeech/hifigan_v2 is already downloaded.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_lengt